In [1]:
# SETUP
#
# Install keras R
# install.packages('keras')
# 
# Update reticulate from cran (it defaults to mran which has an outdated version)
# install.packages("reticulate", repos = "https://cloud.r-project.org")

library(keras)
# use_python('C:\\Anaconda\\envs\\py35')
use_python('/anaconda/envs/py35')

# Import hyper-parameters
library(reticulate)
source_python("./common/params_inf.py")

# Import util functions
source("./common/utils.R")

In [2]:
# py_config()
print(paste0("OS: ", Sys.info()["sysname"]))
print(R.version$version.string)
print(paste0("Keras: ", packageVersion("keras")))
print(paste0("Tensorflow: ", packageVersion("tensorflow")))
print(paste0("Keras using ", backend()$backend()))
print(paste0("Keras channel ordering is ", backend()$image_data_format())) 

[1] "OS: Linux"
[1] "R version 3.4.1 (2017-06-30)"
[1] "Keras: 2.0.6"
[1] "Tensorflow: 1.3.1"
[1] "Keras using tensorflow"
[1] "Keras channel ordering is channels_last"


In [3]:
# Create batches of fake data
fake_data <- give_fake_data(BATCH_SIZE*BATCHES_GPU)
fake_input_data_cl <- fake_data[[1]]

In [4]:
cat('x_train shape:', dim(fake_input_data_cl), '\n')

x_train shape: 1280 224 224 3 


In [5]:
# Return features from classifier
predict_fn <- function(classifier, data, batchsize){
    out <- array(0, dim = c(dim(data)[1], RESNET_FEATURES))
    idx <- 0:(dim(data)[1] %/% batchsize - 1)
    for (i in idx){
        dta <- data[(i*batchsize + 1):((i+1)*batchsize),,,]
        out[(i*batchsize + 1):((i+1)*batchsize), ] <- predict_on_batch(classifier, dta)
    }
    return(out)
}

In [6]:
# instantiate the model
model <- application_resnet50(include_top = FALSE, input_shape = c(224L, 224L, 3L))
#summary(model)

In [7]:
# Time execution 
ptm <- proc.time()
features_cl = predict_fn(model, fake_input_data_cl, BATCH_SIZE)
proc.time() - ptm

   user  system elapsed 
 19.026   2.843  22.386 

In [8]:
# Time execution on channels-first data
print(paste("Images per second ", (BATCH_SIZE*BATCHES_GPU)/22.38))

[1] "Images per second  57.1939231456658"
